In [16]:
import pandas as pd

def calculate_head_to_head(dataset):
    # Convertir la columna 'tourney_date' a formato de fecha
    dataset['tourney_date'] = pd.to_datetime(dataset['tourney_date'], format='%Y%m%d')

    # Ordenar el dataset por fecha
    dataset = dataset.sort_values('tourney_date')

    # Inicializar una nueva columna para el historial de enfrentamientos directos
    dataset['head_to_head'] = -1.0
    '''dataset['num_matches'] = 0'''
    # Iterar sobre las filas del dataset
    for index, row in dataset.iterrows():
        # Obtener los partidos anteriores entre los dos jugadores
        previous_matches = dataset[((dataset['winner_id'] == row['winner_id']) & (dataset['loser_id'] == row['loser_id']) | (dataset['winner_id'] == row['loser_id']) & (dataset['loser_id'] == row['winner_id'])) & (dataset['tourney_date'] < row['tourney_date'])]

        if not previous_matches.empty:
            # Calcular el porcentaje de veces que el 'winner_id' ha ganado
            wins = len(previous_matches[previous_matches['winner_id'] == row['winner_id']])
            total = len(previous_matches)
            dataset.at[index, 'head_to_head'] = wins / total
            '''# Calcular el número de partidos jugados
            dataset.at[index, 'num_matches'] = total'''

    return dataset



In [13]:
# Cargar los datos de los años 2022 y 2023
data_2021 = pd.read_csv('atp_matches_2021.csv')
data_2022 = pd.read_csv('atp_matches_2022.csv')
data_2023 = pd.read_csv('atp_matches_2023.csv')
data_2021_qc = pd.read_csv('atp_matches_qual_chall_2021.csv')
data_2022_qc = pd.read_csv('atp_matches_qual_chall_2022.csv')
data_2023_qc = pd.read_csv('atp_matches_qual_chall_2023.csv')

# Agregar una columna 'source' a cada DataFrame
data_2021['source'] = 'atp_matches_2021.csv'
data_2022['source'] = 'atp_matches_2022.csv'
data_2023['source'] = 'atp_matches_2023.csv'
data_2021_qc['source'] = 'atp_matches_qual_chall_2021.csv'
data_2022_qc['source'] = 'atp_matches_qual_chall_2022.csv'
data_2023_qc['source'] = 'atp_matches_qual_chall_2023.csv'

# Combinar los datos en un solo DataFrame
data_all = pd.concat([data_2021, data_2021_qc, data_2022, data_2022_qc, data_2023, data_2023_qc])

# Aplicar la función calculate_head_to_head
data_all = calculate_head_to_head(data_all)

# Filtrar los datos para obtener solo los partidos de 2023 del archivo 'atp_matches_2023.csv'
data_2023 = data_all[(data_all['tourney_date'].dt.year == 2023) & (data_all['source'] == 'atp_matches_2023.csv')]


In [20]:
dataset = pd.read_csv('atp_matches_qual_chall_2023.csv')
dataset = calculate_head_to_head(dataset)

In [6]:
primera_fecha = dataset['tourney_date'].min()
print(f"La primera fecha en 'tourney_date' es {primera_fecha}")


NameError: name 'dataset' is not defined

In [14]:
data_2023['head_to_head'][data_2023['head_to_head'] != -1.0].count() / len(data_2023)

0.775284661754856

In [15]:
data_2023['num_matches'].max()

9

Problema: la gran mayoría de partidos son primer enfrentamiento. Para que la función head_to_head tenga mayor utilidad habrá que unir los datasets de varios años atrás. Puede ser que para esto haya que tener en cuenta de alguna forma en que año son los enfrentamientos